In [1]:
import tensorflow as tf
import numpy as np
import os
import re
import io


In [2]:
import pickle
data_dir = "/home/maikfangogoair/tmp/save/"
pkl_file = "spectrogram_v2.pkl"
with open (data_dir + pkl_file, 'rb') as fp:
    all_data = pickle.load(fp)

In [25]:
tf.reset_default_graph();
input=tf.placeholder(tf.float32, (None, 99, 161), 'input')
def spectrogram_ConvNet(input):
    """
    """
    dropout_prob = tf.placeholder(tf.float32, name='dropout_prob')
    
    
    reshape_input = tf.reshape(input, [-1, 99, 161, 1], 'reshape_input')
    batch_input = tf.layers.batch_normalization(reshape_input)
    
    conv_1 = tf.layers.conv2d(batch_input, 64, (5,5), (1,1), 'same', name='conv_1')
    relu_1 = tf.nn.relu(conv_1,'relu_1')
    dropout_1 = tf.nn.dropout(relu_1, dropout_prob, name = 'dropout_1')
    
    max_pool = tf.nn.max_pool(dropout_1, [1, 2, 2, 1], [1, 2, 2, 1], 'SAME')

    conv_2 = tf.layers.conv2d(max_pool, 64, (3,3), (1,1), 'same')
    relu_2 = tf.nn.relu(conv_2,'relu_2')
    dropout_2 = tf.nn.dropout(relu_2, dropout_prob, name = 'dropout_2')
    
    size=int(dropout_2.get_shape()[1]) * int(dropout_2.get_shape()[2]) * int(dropout_2.get_shape()[3])
    flatten = tf.reshape(dropout_2, (-1, size), 'flatten')

    logits = tf.layers.dense(flatten, 12,name = 'logits')
    
    return logits,dropout_prob

In [26]:
model_name = "spectrogram_ConvNet"
if model_name == "spectrogram_ConvNet":
    logits,dropout_prob = spectrogram_ConvNet(input)

In [27]:
#training steps
ground_truth_input = tf.placeholder(tf.int64, [None], name='groundtruth_input')
learning_rate_input = tf.placeholder(tf.float32, [], name='learning_rate_input')
cross_entropy_mean = tf.losses.sparse_softmax_cross_entropy(ground_truth_input, logits)
train_step = tf.train.GradientDescentOptimizer(learning_rate_input).minimize(cross_entropy_mean)

predicted_indices = tf.argmax(logits, 1)
correct_prediction = tf.equal(predicted_indices, ground_truth_input)
confusion_matrix = tf.confusion_matrix(ground_truth_input, predicted_indices, num_classes=12)
evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [31]:
import random
random.seed(1222)
size=len([x for x in all_data if x["group"] == "training" ])
saver = tf.train.Saver(tf.global_variables())
init = tf.global_variables_initializer()

batch_size = 300
learning_rate = 1e-4
learning_decay = 0.9
learning_decay_period = 350
eval_every_steps = 100
training_steps = 10000
print_every = 1
eval_size = 300
validation_set = [x["spectrogram"] for x in all_data if x["group"] == "validation"]

with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess, data_dir + 'spectrogram_ConvNet.ckpt-500')
    for i in range(1, training_steps+1):
        start_pos = (i-1) * batch_size % size
        end_pos = (i-1) * batch_size % size + batch_size 
        train_fingerprints = np.asarray([x["spectrogram"] for x in all_data if x["group"] == "training"][start_pos:end_pos])
        train_ground_truth = np.asarray([x["label_idx"] for x in all_data if x["group"] == "training"][start_pos:end_pos]) 
        loss, _ ,accurancy,  = sess.run([cross_entropy_mean, train_step, evaluation_step], \
                feed_dict={
                    input: train_fingerprints,
                    ground_truth_input: train_ground_truth,
                    learning_rate_input: learning_rate * (learning_decay**(i // learning_decay_period)),
                    dropout_prob: 0.5
                })
        if i % print_every == 0:
            print("Step %d, loss: %f, accurancy: %f." % (i, loss, accurancy))
        if  i % eval_every_steps == 0:
            saver.save(sess, data_dir + model_name +".ckpt", global_step=i)
            rand_choice = [random.randint(0,len(validation_set)-1) for x in range(eval_size)]
            _, accurancy = sess.run([train_step,evaluation_step], \
                feed_dict={
                    input: np.asarray(validation_set)[rand_choice],
                    ground_truth_input: np.asarray([x["label_idx"] for x in all_data if x["group"] == "validation"])[rand_choice], \
                    learning_rate_input: learning_rate * (learning_decay**(i // learning_decay_period)),
                    dropout_prob: 1.0
                })
            print("Validation accurancy is %f" % accurancy)

INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/spectrogram_ConvNet.ckpt-500


INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/spectrogram_ConvNet.ckpt-500


Step 1, loss: 1.267259, accurancy: 0.583333.
Step 2, loss: 1.164033, accurancy: 0.646667.
Step 3, loss: 1.352372, accurancy: 0.590000.
Step 4, loss: 1.190791, accurancy: 0.653333.
Step 5, loss: 1.233237, accurancy: 0.606667.
Step 6, loss: 1.246054, accurancy: 0.636667.
Step 7, loss: 1.331899, accurancy: 0.606667.
Step 8, loss: 1.184370, accurancy: 0.646667.
Step 9, loss: 1.257778, accurancy: 0.573333.
Step 10, loss: 1.360212, accurancy: 0.583333.
Step 11, loss: 1.185382, accurancy: 0.616667.
Step 12, loss: 1.336402, accurancy: 0.556667.
Step 13, loss: 1.169189, accurancy: 0.610000.
Step 14, loss: 1.266326, accurancy: 0.566667.
Step 15, loss: 1.112651, accurancy: 0.653333.
Step 16, loss: 1.102832, accurancy: 0.650000.
Step 17, loss: 1.288378, accurancy: 0.590000.
Step 18, loss: 1.323243, accurancy: 0.613333.
Step 19, loss: 1.216810, accurancy: 0.623333.
Step 20, loss: 1.291352, accurancy: 0.616667.
Step 21, loss: 1.286364, accurancy: 0.613333.
Step 22, loss: 1.181442, accurancy: 0.61333

KeyboardInterrupt: 

In [14]:
#testing
with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, data_dir + 'spectrogram_ConvNet.ckpt-1600')
    predicted_list = sess.run(predicted_indices, \
        feed_dict={
            input: np.asarray([x["spectrogram"] for x in all_data if x["group"] == "testing"]) ,
            dropout_prob: 1.0
        })

INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/spectrogram_ConvNet.ckpt-1600


INFO:tensorflow:Restoring parameters from /home/maikfangogoair/tmp/save/spectrogram_ConvNet.ckpt-1600


In [51]:
predicted_list_part = np.hstack((predicted_list_part1,predicted_list_part2,predicted_list_part3))

predicted_list_part

correct_idx = np.asarray([x["label_idx"] for x in all_data if x["group"] == "testing"])

wrong_cnt = 0
for i in range(correct_idx.shape[0]):
    if predicted_list_part[i] != correct_idx[i]:
        wrong_cnt += 1
wrong_cnt

(2979 - 1255)/2979

test_data = [x for x in all_data if x["group"] == "testing"]
test_data[1]

wrong_data = []
word_list = ['silence','unknown', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
for i in range(correct_idx.shape[0]):
    if predicted_list_part[i] != correct_idx[i]:
        test_data[i]["predicted_label"] = word_list[predicted_list_part[i]]
        wrong_data.append(test_data[i])

#which pair most confusing?
confuse_map = {}
for x in wrong_data:
    key = x["label"] + " is confused by " + x["predicted_label"]
    cnt = confuse_map.get(key, 0)
    confuse_map[key] = cnt + 1 
#https://stackoverflow.com/questions/613183/how-do-i-sort-a-dictionary-by-value
import operator
sorted_confuse_map = sorted(confuse_map.items(), key=operator.itemgetter(1), reverse=True)
sorted_confuse_map